In [10]:
from utils import *

# Minigame round differences

In [11]:
data = pd.read_csv(os.path.join('..', 'data', 'minigame_results.csv'), delimiter=';')
data = data[data.respondent_id != 1].reset_index(drop=True)
data.note = data.note.map(lambda x: x[x.find('_')+1:])
data.head()

,id,session_id,points,hits,game_number,creation,id.1,respondent_id,note,prototype,creation.1,id.2,experience,pc_usage,sex,age,employment,creation.2
0,16,6,115,17,1,2022-11-26 11:10:29.358,6,2,micr1000_sys10,1A,2022-11-26 11:09:47.85,2,5,2 - 3 hours daily,man,29,učiteľ,2022-11-26 11:09:47.721
1,17,6,100,16,2,2022-11-26 11:10:47.334,6,2,micr1000_sys10,1A,2022-11-26 11:09:47.85,2,5,2 - 3 hours daily,man,29,učiteľ,2022-11-26 11:09:47.721
2,18,6,107,15,3,2022-11-26 11:11:05.046,6,2,micr1000_sys10,1A,2022-11-26 11:09:47.85,2,5,2 - 3 hours daily,man,29,učiteľ,2022-11-26 11:09:47.721
3,19,7,69,15,1,2022-11-26 11:15:57.292,7,2,dell2400_sys5,2A,2022-11-26 11:15:24.863,2,5,2 - 3 hours daily,man,29,učiteľ,2022-11-26 11:09:47.721
4,20,7,74,15,2,2022-11-26 11:16:17.721,7,2,dell2400_sys5,2A,2022-11-26 11:15:24.863,2,5,2 - 3 hours daily,man,29,učiteľ,2022-11-26 11:09:47.721


In [12]:
data['order'] = -1
for window in data.rolling(15):
    if(len(window) == 15 and window.index.values[0] % 15 == 0):
        data.loc[window.index.values, 'order'] = [0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4]

In [4]:
my_shapiro(data, ['hits', 'points'], verbose=True)

hits:
  p-value: <0.0005 
points:
  p-value: 0.14203232526779175 
1/2 normally distributed metrics.


## Points and hits differences

In [5]:
data[['hits', 'points', 'game_number']].groupby(['game_number']).agg(
    MeanHits=('hits', np.mean),
    StdHits=('hits', np.std),
    MeanPoints=('points', np.mean),
    StdPoints=('points', np.std),
)

,MeanHits,StdHits,MeanPoints,StdPoints
game_number,,,,
1,13.2937,2.7256,76.3375,25.6787
2,14.0188,2.5929,84.7875,25.6013
3,14.4563,2.5865,87.8875,26.4836


In [6]:
my_friedman(data, ['hits', 'points'], 'game_number', [1, 2, 3])

,Feature name,p,Q,effsize,significant,1__2,1__3,2__3
0,hits,< .001,77.4600,0.24,True,< .001,< .001,0.032
1,points,< .001,57.9600,0.18,True,< .001,< .001,0.214


In [7]:
my_kruskal(data, ['hits', 'points'], 'game_number', [1, 2, 3])

,feature,p_value,F_statistic,effsize,significant,1__2,1__3,2__3
0,hits,< .001,15.3100,0.03,True,0.011,< .001,0.191
1,points,0.001,13.1900,0.02,True,0.009,< .001,0.381


### Ordered rounds

In [15]:
my_friedman(data[data.order == 0], ['hits', 'points'], 'game_number', [1, 2, 3])

,Feature name,p,Q,effsize,significant,1__2,1__3,2__3
0,hits,< .001,22.5300,0.35,True,0.009,< .001,0.322
1,points,< .001,23.1600,0.36,True,0.001,< .001,0.536


In [16]:
data[data.order == 0][['hits', 'points', 'game_number']].groupby(['game_number']).agg(
    MeanHits=('hits', np.mean),
    StdHits=('hits', np.std),
    MeanPoints=('points', np.mean),
    StdPoints=('points', np.std),
)

,MeanHits,StdHits,MeanPoints,StdPoints
game_number,,,,
1,12.2812,3.4570,71.5938,31.8868
2,13.5312,3.1003,84.2500,29.4947
3,14.3125,2.9669,88.2188,30.2865


In [17]:
my_friedman(data[data.order == 4], ['hits', 'points'], 'game_number', [1, 2, 3])

,Feature name,p,Q,effsize,significant,1__2,1__3,2__3
0,hits,< .001,17.7800,0.28,True,0.054,< .001,0.355
1,points,0.325,2.2500,0.04,False,None,None,None


In [18]:
data[data.order == 4][['hits', 'points', 'game_number']].groupby(['game_number']).agg(
    MeanHits=('hits', np.mean),
    StdHits=('hits', np.std),
    MeanPoints=('points', np.mean),
    StdPoints=('points', np.std),
)

,MeanHits,StdHits,MeanPoints,StdPoints
game_number,,,,
1,13.8750,2.5747,84.0000,24.4316
2,14.5312,2.7590,87.8750,30.7956
3,15.0312,2.6577,92.1875,29.4152


In [19]:
my_friedman(data[data.order == 2], ['hits', 'points'], 'game_number', [1, 2, 3])

,Feature name,p,Q,effsize,significant,1__2,1__3,2__3
0,hits,0.005,10.4200,0.16,True,0.235,0.02,0.536
1,points,0.003,11.6300,0.18,True,0.02,0.005,0.893


### Individual mice

In [39]:
configs = {'micr1000_sys10' : [0,4], 'dell2400_sys5' : [1,3], 'dell2400_sys10':None, 'dell2400_sys18' : [1,3],'trust4800_sys10': [0,4]}

In [40]:
for config in configs:
    print(config)
    print(my_kruskal(data[data.note == config], ['hits', 'points'], 'game_number', [1, 2, 3]), end='\n\n')

micr1000_sys10
  feature p_value  F_statistic effsize  significant   1__2   1__3   2__3
0    hits   0.086       4.9000    0.03        False   None   None   None
1  points   0.028       7.1500    0.06         True  0.038  0.012  0.668

dell2400_sys5
  feature p_value  F_statistic effsize  significant  1__2   1__3   2__3
0    hits    0.21       3.1200    0.01        False  None   None   None
1  points   0.018       8.0100    0.06         True  0.06  0.006  0.373

dell2400_sys10
  feature p_value  F_statistic effsize  significant  1__2  1__3  2__3
0    hits    0.31       2.3400     0.0        False  None  None  None
1  points   0.333       2.2000     0.0        False  None  None  None

dell2400_sys18
  feature p_value  F_statistic effsize  significant  1__2  1__3  2__3
0    hits   0.321       2.2700     0.0        False  None  None  None
1  points   0.335       2.1800     0.0        False  None  None  None

trust4800_sys10
  feature p_value  F_statistic effsize  significant   1__2   1__3 

In [41]:
for config in configs:
    print(config)
    print(data[(data.note==config)][['hits', 'points', 'game_number']].groupby(['game_number']).agg(
        MeanHits=('hits', np.mean),
        StdHits=('hits', np.std),
        MeanPoints=('points', np.mean),
        StdPoints=('points', np.std),
    ), end='\n\n')

micr1000_sys10
             MeanHits  StdHits  MeanPoints  StdPoints
game_number                                          
1             14.4062   2.7340     95.0000    23.1056
2             15.3438   2.7073    106.6875    24.8887
3             15.8438   2.6410    110.1875    24.1052

dell2400_sys5
             MeanHits  StdHits  MeanPoints  StdPoints
game_number                                          
1             14.0312   2.7531     82.5000    22.2174
2             14.6250   2.4593     93.4375    22.2681
3             15.1875   2.6081     98.7812    23.6421

dell2400_sys10
             MeanHits  StdHits  MeanPoints  StdPoints
game_number                                          
1             13.3750   2.2107     76.4062    22.8259
2             13.8125   2.0547     83.5312    20.8404
3             14.1562   2.1419     85.7500    22.7171

dell2400_sys18
             MeanHits  StdHits  MeanPoints  StdPoints
game_number                                          
1             12.906

In [44]:
for order in [0, 1, 2, 3, 4]:
    for config in configs:
        res = my_kruskal(data[(data.order == order) & (data.note == config)], ['hits', 'points'], 'game_number', [1, 2, 3])
        if(res.loc[0, 'p_value'] != 'nan'):
            print('order:', order, '  config:', config)
            print(res, end='\n\n')

order: 0   config: micr1000_sys10
  feature p_value  F_statistic effsize  significant  1__2  1__3  2__3
0    hits    0.42       1.7300   -0.01        False  None  None  None
1  points   0.425       1.7100   -0.01        False  None  None  None

order: 0   config: trust4800_sys10
  feature p_value  F_statistic effsize  significant   1__2   1__3   2__3
0    hits   0.006      10.2100    0.18         True  0.052  0.002  0.222
1  points   0.005      10.4600    0.19         True   0.01  0.003  0.672

order: 1   config: dell2400_sys5
  feature p_value  F_statistic effsize  significant  1__2  1__3  2__3
0    hits   0.553       1.1800   -0.02        False  None  None  None
1  points    0.27       2.6200    0.01        False  None  None  None

order: 1   config: dell2400_sys18
  feature p_value  F_statistic effsize  significant  1__2  1__3  2__3
0    hits   0.291       2.4700    0.01        False  None  None  None
1  points   0.112       4.3700    0.05        False  None  None  None

order: 2   

### Results across trials (out of 5) for each mouse

In [46]:
for config in configs:
    if(configs[config]):
        print(config)
        print()
        print(my_mannwhitney(data[(data.note==config)], ['hits', 'points'], 'order', configs[config]), end='\n\n')

micr1000_sys10

  Feature name      p          U       z  effsize  significant
0         hits  0.416 1,041.5000 -0.8100  -0.0800        False
1       points    0.7 1,099.0000 -0.3800  -0.0400        False

dell2400_sys5

  Feature name      p          U       z  effsize  significant
0         hits  0.368 1,274.5000  0.9000   0.0900        False
1       points  0.315 1,014.5000 -1.0000  -0.1000        False

dell2400_sys18

  Feature name      p          U       z  effsize  significant
0         hits  0.012   811.0000 -2.5000  -0.2500         True
1       points  0.875 1,174.0000  0.1600   0.0200        False

trust4800_sys10

  Feature name       p        U       z  effsize  significant
0         hits  < .001 685.5000 -3.4100  -0.3500         True
1       points   0.007 783.0000 -2.7000  -0.2800         True



In [52]:
for config in configs:
    if(configs[config]):
        for round in [1, 2, 3]:
            print('config: ', config, '   round: ', round)
            print()
            print(my_mannwhitney(data[(data.note==config) & (data.game_number==round)], ['hits', 'points'], 'order', configs[config]), end='\n\n')

config:  micr1000_sys10    round:  1

  Feature name      p        U       z  effsize  significant
0         hits  0.985 127.0000 -0.0200  -0.0000        False
1       points  0.985 129.0000  0.0600   0.0100        False

config:  micr1000_sys10    round:  2

  Feature name      p        U       z  effsize  significant
0         hits  0.675 116.5000 -0.4100  -0.0700        False
1       points   0.72 118.0000 -0.3600  -0.0600        False

config:  micr1000_sys10    round:  3

  Feature name      p        U       z  effsize  significant
0         hits  0.372 104.0000 -0.8900  -0.1600        False
1       points  0.637 115.0000 -0.4700  -0.0800        False

config:  dell2400_sys5    round:  1

  Feature name      p        U       z  effsize  significant
0         hits  0.519 145.5000  0.6800   0.1200        False
1       points   0.91 124.5000 -0.1100  -0.0200        False

config:  dell2400_sys5    round:  2

  Feature name      p        U       z  effsize  significant
0         hits 

In [47]:
for config in configs:
    print(config)
    print(data[(data.note==config)][['hits', 'points', 'order', 'game_number']].groupby(['order', 'game_number']).agg(
        MeanHits=('hits', np.mean),
        StdHits=('hits', np.std),
        MeanPoints=('points', np.mean),
        StdPoints=('points', np.std),
    ), end='\n\n')

micr1000_sys10
                   MeanHits  StdHits  MeanPoints  StdPoints
order game_number                                          
0     1             14.4375   3.1192     95.5000    25.0945
      2             15.2500   2.9326    104.1250    26.7927
      3             15.5625   2.9658    107.0000    26.4651
4     1             14.3750   2.3910     94.5000    21.7501
      2             15.4375   2.5552    109.2500    23.4165
      3             16.1250   2.3345    113.3750    21.8811

dell2400_sys5
                   MeanHits  StdHits  MeanPoints  StdPoints
order game_number                                          
1     1             14.3125   3.2397     81.0625    24.3707
      2             14.6875   2.9148     88.9375    25.1010
      3             15.4375   3.1616     94.1875    25.1030
3     1             13.7500   2.2361     83.9375    20.5377
      2             14.5625   1.9990     97.9375    18.7491
      3             14.9375   1.9822    103.3750    21.9085

dell2400_

### Points - anova

In [21]:
my_shapiro(data[data.game_number == 1], ['points'], True)
my_shapiro(data[data.game_number == 2], ['points'], True)
my_shapiro(data[data.game_number == 3], ['points'], True)

points:
  p-value: 0.10479385405778885 
1/1 normally distributed metrics.
points:
  p-value: 0.6996892690658569 
1/1 normally distributed metrics.
points:
  p-value: 0.22773946821689606 
1/1 normally distributed metrics.


In [22]:
print(AnovaRM(data=data[['session_id', 'points', 'game_number']].reset_index(drop=True).reset_index(drop=True), depvar='points', subject='session_id', within=['game_number']).fit())
print('1vs2:', 2.25031e-12)
print('1vs3:', 1.33227e-15)
print('2vs3:', 0.00638247)  

                  Anova
            F Value Num DF  Den DF  Pr > F
------------------------------------------
game_number 51.3977 2.0000 318.0000 0.0000

1vs2: 2.25031e-12
1vs3: 1.33227e-15
2vs3: 0.00638247


## Errors

In [23]:
data_raw = pd.read_csv(os.path.join('..', 'data', 'mouseLogs_minigame.csv'), delimiter=';')

data_raw = data_raw[(data_raw['event'] != 'mousemove')]
data_raw = data_raw.sort_values(by=['respondent_id', 'session_id', 'game_number', 'timestamp'])
data_grouped = data_raw.groupby(['respondent_id', 'session_id', 'game_number'])

hits_and_errors = pd.DataFrame(columns=['respondent_id', 'session_id', 'game_number', 'note', 'sex', 'calc_hits', 'errors'])

for name, group in data_grouped :
    hits = 1
    errors = 0
    next_target_number = group.head(1)['next_target_number'].values[0]

    for i, item in group.iterrows():
        if (item['event'] == 'mouseup'):
            if (item['next_target_number'] == next_target_number + 1):
                hits += 1
                next_target_number += 1
            elif(item['next_target_number'] == next_target_number):
                errors += 1
    row = [name[0], name[1], name[2], group['note'].max(), group['sex'].max(), hits, errors]
    hits_and_errors.loc[len(hits_and_errors)] = row

my_friedman(hits_and_errors, ['errors'], 'game_number', [1, 2, 3])

,Feature name,p,Q,effsize,significant,1__2,1__3,2__3
0,errors,0.955,0.0900,0.0,False,None,None,None
